## Import Libraries

In [40]:
import time
import rospy
import numpy as np
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from dual_quaternion import Quaternion
from dual_quaternion import DualQuaternion
from casadi import Function

## Working with Symbolic Variables

In [41]:
# Defining First transformation
theta_1 = ca.MX.sym('theta_1', 1, 1)
n_1 = ca.MX([0.0, 0.0, 1.0])
q_1 = ca.vertcat(ca.cos(theta_1/2), ca.sin(theta_1/2)@n_1)
t_1 = ca.MX([0.0, 0.0, 0.0, 0.0])

# Defining second transformation
theta_2 = ca.MX.sym('theta_2', 1, 1)
n_2 = ca.MX([0.0, 0.0, 1.0])
q_2 = ca.vertcat(ca.cos(theta_2/2), ca.sin(theta_2/2)@n_2)
tx_2 = ca.MX.sym("x_2", 1, 1)
t_2 = ca.vertcat(0.0, tx_2, 0.0, 0.0)

# Defining third transformation
theta_3 = ca.MX.sym('theta_3', 1, 1)
n_3 = ca.MX([0.0, 0.0, 1.0])
q_3 = ca.vertcat(ca.cos(theta_3/2), ca.sin(theta_3/2)@n_3)
tx_3 = ca.MX.sym("x_3", 1, 1)
t_3 = ca.vertcat(0.0, tx_3, 0.0, 0.0)

aux_symbolic_total = ca.vertcat(theta_1, theta_2, tx_2, theta_3, tx_3)
symbolic_actions = ca.vertcat(theta_1, theta_2, theta_3)

In [42]:
Q1 = DualQuaternion.from_pose(quat = q_1, trans = t_1)
Q2 = DualQuaternion.from_pose(quat = q_2, trans = t_2)
Q3 = DualQuaternion.from_pose(quat = q_3, trans = t_3)

In [43]:
Q4 = Q1 * Q2 * Q3

In [44]:
Function_DualQuat_total = Function('DualQuat_total', [aux_symbolic_total], [Q4.get[:, 0]])
Function_DualQuat_total_trans = Function('DualQuat_total_trans', [aux_symbolic_total], [Q4.get_trans.get[:, 0]])
Function_DualQuat_total_quat = Function('DualQuat_total_quat', [aux_symbolic_total], [Q4.get_quat.get[:, 0]])

In [45]:
input_t = np.array([np.pi/2, np.pi/2, 2, -np.pi/4, 3])

trans_t = Function_DualQuat_total_trans(input_t)
quat_t = Function_DualQuat_total_quat(input_t)
Quat_t = Function_DualQuat_total(input_t)

In [46]:
trans_t

DM([-0, -3, 2, 0])

In [47]:
quat_t

DM([0.382683, 0, 0, 0.92388])

In [48]:
Quat_t

DM([0.382683, 0, 0, 0.92388, -0, 0.349854, 1.7685, 0])

### Calculating Jacobians

In [50]:
J = ca.jacobian(Q4.get[:, 0], symbolic_actions)

In [51]:
Function_J = Function("Function_F", [aux_symbolic_total], [J])

In [52]:
J.shape

(8, 3)